In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bracketology_rl import Bracket
from bracketgym1game import TournamentEnv
import warnings
import gym
import numpy as np
from tourney_tools import * 
warnings.filterwarnings('ignore')
import multiprocessing as mp
import traceback


In [35]:
conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia

In [7]:
from stable_baselines3 import PPO,A2C,SAC,DQN,DDPG
from sb3_contrib import RecurrentPPO,ARS,QRDQN
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement
from stable_baselines3.common.env_util import make_vec_env
import torch
from torch.utils.data import DataLoader, Dataset

In [8]:
import tensorboard
## tensor board_log
tensorboard_log = "Logs/"

In [9]:
years = list(range(2003,2019))
year=years[1]

bracket = Bracket(2023)

team_data=pd.read_csv('Process_data/pm_w_names.csv')

In [24]:
env=TournamentEnv(team_data=team_data,weight_scores=True,discrete=False,loading_bar=True)
ensemble=[A2C,ARS]


  0%|          | 0/100000 [00:00<?, ?it/s]

In [25]:

def make_ars_model(model,env):
    policies=[p for p in model.policy_aliases]
    pol=policies[0]
    print(policies,policies[0])
    model = model(policy=pol, env=env, verbose=0,
    learning_rate= .001,
    # gamma=  0.99,

    # exploration_fraction=1,
    tensorboard_log='Logs/')

    return model



In [26]:
env=TournamentEnv(team_data=team_data,weight_scores=True,discrete=False,loading_bar=False)
models=[]
# for model in ensembleQs:
#     models.append(make_Q_model(model,env) )
for model in ensemble:
    models.append(make_ars_model(model,env) )



['MlpPolicy', 'CnnPolicy', 'MultiInputPolicy'] MlpPolicy


RuntimeError: Calling torch.geqrf on a CPU tensor requires compiling PyTorch with LAPACK. Please use PyTorch built with LAPACK support.

## Train the Models ->2019

In [ ]:
best_params=[]
bar=tqdm(range(len(models)))

for i,model in enumerate(models):
    bar.update(1)
    try:
        stop_train_callback = StopTrainingOnNoModelImprovement(max_no_improvement_evals=1000, min_evals=1000, verbose=0)
        eval_callback = EvalCallback(model.env, eval_freq=20, callback_after_eval=stop_train_callback, verbose=0)
        model.learn(5000,callback=eval_callback)
    except KeyboardInterrupt:

        continue

## test the ensembled models on 2022

In [ ]:
env=TournamentEnv(team_data=team_data,discrete=False,weight_scores=False,loading_bar=True,bust_stop=False)
## test as an ensemble
scores=[]

for model in models:
    model.set_env(env)


In [ ]:
## test one 2022
obs = env.reset(season=2022,weight_scores=False,track_wins=True)
done = False
while not done:
    actions = [model.predict(obs,deterministic =True)[0] for model in models]
    action=np.mean(actions, axis=0)

    obs, reward, done, info = env.step(action)
display(info['total_score'],info['num_correct'])    
env.render(score=f"Ensembled ARS Correct:{info['num_correct']} {info['total_score']}")



In [ ]:
## test individually
scores=[]
max_scores=[]
model_scores={}
dfs={}
for i,model in enumerate(models):

    obs = env.reset(season=2022,weight_scores=False,track_wins=False)
    done = False
    while not done:
        action = model.predict(obs,deterministic=False)[0]

        obs, reward, done, info = env.step(action)
    total_score=info['total_score']
    num_correct=info['num_correct']
    scores.append(total_score) 
    dfs[total_score]=env.action_data
    model_scores[total_score]=model
    name=model.__repr__().split(' ')[0].split('.')[-1]
    env.render(score=f'{name} {i} Correct:{num_correct} Score:{total_score}')
display(info['total_score'],info['num_correct'])    


In [ ]:
## pick out the best Models
scores=sorted(scores)
topscore=scores[-1]
secondscore=scores[-3]
print(scores)
top_model=model_scores[topscore]
second_model=model_scores[secondscore]


## predict This year

In [ ]:

obs = env.reset(season=2023,weight_scores=False,track_wins=True)
done = False
while not done:
    actions = [model.predict(obs,deterministic =False)[0] for model in models]
    action=np.mean(actions, axis=0)

    obs, reward, done, info = env.step(action)

env.render(score=f"Ensembled ARS Prediction")

In [ ]:

obs = env.reset(season=2023,weight_scores=False,track_wins=False)
done = False
while not done:
    action = top_model.predict(obs,deterministic=False)[0]
    obs, reward, done, info = env.step(action)
name=top_model.__repr__().split(' ')[0].split('.')[-1]
env.render(score=f'Top {name}  Pred')

In [ ]:
obs = env.reset(season=2023,weight_scores=False,track_wins=False)
done = False
while not done:
    action = second_model.predict(obs,deterministic=False)[0]
    obs, reward, done, info = env.step(action)
name=second_model.__repr__().split(' ')[0].split('.')[-1]
env.render(score=f'Second Best {name} Pred')

In [ ]:
# for i,model in enumerate(models):

#     obs = env.reset(season=2023,weight_scores=False,track_wins=False)
#     done = False
#     while not done:
#         action = model.predict(obs,deterministic=True)[0]

#         obs, reward, done, info = env.step(action)
#     env.render(score=f'Model {i} Pred')


jgfulfgu